In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
from vmdpy import VMD  
import pandas as pd
from scipy.fftpack import fft

In [ ]:
# Load the Excel file
filename = '\Runoff_SHENYANG.xlsx'
# Read the desired column from the Excel file
f = pd.read_excel(filename, usecols=[3])
# Plot the data
plt.plot(f.values)
plt.show()
print(np.min(f.values))
# 输出该列的数据数量
print("数据总数：", len(f.values))
print(f.values)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

filename = '\Runoff_SHENYANG.xlsx'
# Read the desired column from the Excel file
f = pd.read_excel(filename, usecols=[3])
# 过滤掉值为 0 和 NaN 的数据
f_filtered = f[(f.values != 0) & (~pd.isna(f.values))]
# Set font to Times New Roman globally
plt.rcParams['font.family'] = 'Times New Roman'
# 设置画布大小
fig = plt.figure(figsize=(12, 2), dpi=200)  # 可以根据需要调整宽度和高度

# Add a global label for the y-axis (Streamflow)
fig.text(0.08, 0.5, 'Streamflow (m³/day)', fontsize=10, va='center', rotation='vertical')
# Add a global label for the x-axis (Times)
fig.text(0.5, -0.05, 'Times (day)', fontsize=10, ha='center')

plt.plot(f_filtered.values, c='#08D0FC')
plt.show()

print("最小值：", np.min(f_filtered.values))
print("过滤后数据总数：", len(f_filtered))


In [ ]:
alpha = 26000        # moderate bandwidth constraint  
tau = 0.3            # noise-tolerance (no strict fidelity enforcement)  
K = 14               # 14 modes  
DC = 0               # no DC part imposed  
init = 1             # initialize omegas uniformly  
tol = 1e-7  
"""  
alpha、tau、K、DC、init、tol 六个输入参数的无严格要求； 
alpha 带宽限制 经验取值为 抽样点长度 1.5-2.0 倍； 
tau 噪声容限 ；
K 分解模态（IMF）个数； 
DC 合成信号若无常量，取值为 0；若含常量，则其取值为 1； 
init 初始化 w 值，当初始化为 1 时，均匀分布产生的随机数； 
tol 控制误差大小常量，决定精度与迭代次数
"""

In [ ]:
u, u_hat, omega = VMD(f.values, alpha, tau, K, DC, init, tol)  
plt.figure()

plt.plot(u.T)
plt.title('Decomposed modes')

In [ ]:
print(f.shape)

In [ ]:
print(u.shape)

In [ ]:
fig1 = plt.figure()
#plt.plot(f.values)

fig1.suptitle('Original input signal and its components')


In [ ]:
# Set font to Times New Roman globally
plt.rcParams['font.family'] = 'Times New Roman'
# Create a figure and set of subplots
fig, axs = plt.subplots(K, 1, figsize=(7, 7), dpi=200)  # Increase figure height for readability
# Loop through each subplot
for i in range(K):
    axs[i].plot(u[i, :], linewidth=0.5, c='#08D0FC')  # Thicker line
    axs[i].set_ylabel(f'IMF{i+1}', fontsize=6, rotation=0, va='center')
    axs[i].tick_params(labelsize=6)  # Adjust tick label size
    # Manually align the ylabel positions
    axs[i].yaxis.set_label_coords(-0.06, 0.5)  # Set the position of the label to the same value for all plots
    if i < K - 1:  # For all subplots except the last one
         axs[i].tick_params(labelbottom=False, bottom=False)  # Hide x-axis labels
# Add a global label for the y-axis (Streamflow)
fig.text(0.04, 0.5, 'Streamflow (m³/day)', fontsize=8, va='center', rotation='vertical')
# Add a global label for the x-axis (Times)
fig.text(0.5, 0.07, 'Times (day)', fontsize=8, ha='center')
# Remove spaces between plots by adjusting subplots layout
plt.subplots_adjust(hspace=0.2)
# Show the figure
plt.show()

In [ ]:
# Set font to Times New Roman globally
plt.rcParams['font.family'] = 'Times New Roman'
# Create a figure and set of subplots
fig, axs = plt.subplots(K, 1, figsize=(7/2, 7), dpi=200)  # Increase figure height for readability
# Loop through each subplot
for i in range(K):
    fft_result = np.abs(np.fft.fft(u[i, :]))  # Compute the FFT
    half_fft_result = fft_result[:len(fft_result) // 2]  # Take the left half of the FFT result
    axs[i].plot(half_fft_result, linewidth=0.5, c='#08D0FC')  # Thicker line for left half of FFT
    # axs[i].plot(abs(fft(u[i,:])), linewidth=0.5, c='#FF5733')  # Thicker line
    axs[i].set_ylabel(f'IMF{i+1}', fontsize=6, rotation=0, va='center')
    axs[i].tick_params(labelsize=6)  # Adjust tick label size
    # Manually align the ylabel positions
    axs[i].yaxis.set_label_coords(-0.15, 0.5)  # Set the position of the label to the same value for all plots
    if i < K - 1:  # For all subplots except the last one
         axs[i].tick_params(labelbottom=False, bottom=False)  # Hide x-axis labels
# Add a global label for the y-axis (Streamflow)
fig.text(-0.05, 0.5, 'Spectral amplitude', fontsize=8, va='center', rotation='vertical')
# Add a global label for the x-axis (Times)
fig.text(0.5, 0.075, 'Frequency (Hz)', fontsize=8, ha='center')
# Remove spaces between plots by adjusting subplots layout
plt.subplots_adjust(hspace=0.2)
# Show the figure
plt.show()

In [ ]:
final_dataframe = pd.DataFrame()
for i in range(K):
    a = u[i, :]  # 
    dataframe = pd.DataFrame({'v{}'.format(i+1): a})  # 
    final_dataframe = pd.concat([final_dataframe, dataframe], axis=1)  #

# save
final_dataframe.to_csv("\VMD_ShenYang.csv", index=False, sep=',')